In [1]:
import heapq
import itertools

class PuzzleNode:
    def __init__(self, state, parent=None, move=None, depth=0):
        self.state = state
        self.parent = parent
        self.move = move
        self.depth = depth
        self.heuristic = self.calculate_heuristic()

    def calculate_heuristic(self):
        # Manhattan distance heuristic
        distance = 0
        for i in range(3):
            for j in range(3):
                value = self.state[i][j]
                if value != 0:
                    goal_row, goal_col = divmod(value - 1, 3)
                    distance += abs(i - goal_row) + abs(j - goal_col)
        return distance + self.depth

    def __lt__(self, other):
        return self.heuristic < other.heuristic

def get_blank_position(state):
    for i in range(3):
        for j in range(3):
            if state[i][j] == 0:
                return i, j

def get_neighbors(node):
    i, j = get_blank_position(node.state)
    neighbors = []
    moves = [(0, 1), (1, 0), (0, -1), (-1, 0)]

    for move in moves:
        ni, nj = i + move[0], j + move[1]
        if 0 <= ni < 3 and 0 <= nj < 3:
            new_state = [list(row) for row in node.state]
            new_state[i][j], new_state[ni][nj] = new_state[ni][nj], new_state[i][j]
            neighbors.append(PuzzleNode(new_state, node, move, node.depth + 1))

    return neighbors

def is_goal(state):
    return all(state[i][j] == i * 3 + j + 1 for i in range(3) for j in range(3))

def print_solution(node):
    moves = []
    while node.parent is not None:
        moves.append(node.move)
        node = node.parent
    moves.reverse()

    for move in moves:
        print(move)

def a_star(initial_state):
    initial_node = PuzzleNode(initial_state)
    heap = [initial_node]
    visited = set()

    while heap:
        current_node = heapq.heappop(heap)

        if is_goal(current_node.state):
            print("Solution found!")
            print_solution(current_node)
            return

        visited.add(tuple(itertools.chain(*current_node.state)))

        neighbors = get_neighbors(current_node)
        for neighbor in neighbors:
            if tuple(itertools.chain(*neighbor.state)) not in visited:
                heapq.heappush(heap, neighbor)

    print("No solution found.")

if __name__ == "__main__":
    # Example initial state
    initial_state = [
        [1, 2, 3],
        [4, 0, 5],
        [7, 8, 6]
    ]

    a_star(initial_state)


No solution found.


In [2]:
from collections import deque

class State:
    def __init__(self, jug_5, jug_7):
        self.jug_5 = jug_5
        self.jug_7 = jug_7

    def __eq__(self, other):
        return self.jug_5 == other.jug_5 and self.jug_7 == other.jug_7

    def __hash__(self):
        return hash((self.jug_5, self.jug_7))

    def __str__(self):
        return f"({self.jug_5}, {self.jug_7})"

def water_jug_bfs(target):
    initial_state = State(0, 0)
    visited = set()
    queue = deque([initial_state])

    while queue:
        current_state = queue.popleft()

        if current_state.jug_7 == target:
            print(f"Target of {target} gallons achieved!")
            return

        visited.add(current_state)

        # Fill jug 5
        fill_jug_5 = State(5, current_state.jug_7)
        if fill_jug_5 not in visited:
            queue.append(fill_jug_5)

        # Fill jug 7
        fill_jug_7 = State(current_state.jug_5, 7)
        if fill_jug_7 not in visited:
            queue.append(fill_jug_7)

        # Pour water from jug 5 to jug 7
        pour_5_to_7 = State(max(0, current_state.jug_5 - (7 - current_state.jug_7)), min(7, current_state.jug_7 + current_state.jug_5))
        if pour_5_to_7 not in visited:
            queue.append(pour_5_to_7)

        # Pour water from jug 7 to jug 5
        pour_7_to_5 = State(min(5, current_state.jug_5 + current_state.jug_7), max(0, current_state.jug_7 - (5 - current_state.jug_5)))
        if pour_7_to_5 not in visited:
            queue.append(pour_7_to_5)

        # Empty jug 5
        empty_jug_5 = State(0, current_state.jug_7)
        if empty_jug_5 not in visited:
            queue.append(empty_jug_5)

        # Empty jug 7
        empty_jug_7 = State(current_state.jug_5, 0)
        if empty_jug_7 not in visited:
            queue.append(empty_jug_7)

    print("Target cannot be achieved with the given jugs.")

if __name__ == "__main__":
    target = 4
    water_jug_bfs(target)


Target of 4 gallons achieved!
